In [1]:
!pip install tensorflow

     |████████████████████████████████| 421.8MB 25kB/s  eta 0:00:01    |██▉                             | 37.1MB 819kB/s eta 0:07:50
     |████████████████████████████████| 26.1MB 290kB/s eta 0:00:01
     |████████████████████████████████| 450kB 389kB/s eta 0:00:01
     |████████████████████████████████| 3.8MB 411kB/s eta 0:00:01
     |████████████████████████████████| 81kB 1.0MB/s eta 0:00:011
     |████████████████████████████████| 163kB 664kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/05/57/ce2e7a8fa7c0afb54a0581b14a65b56e62b5759dbc98e80627142b8a3704/oauthlib-3.1.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/62/1e/a94a8d635fa3ce4cfc7f506003548d0a2447ae76fd5ca53932970fe3053f/pyasn1-0.4.8-py2.py3-none-any.whl
ERROR: tensorflow-gpu 2.0.0 has requirement tensorboard<2.1.0,>=2.0.0, but you'll have tensorboard 2.1.0 which is incompatible.
ERROR: tensorflow-gpu 2.0.0 has requirement tensorflow-estimator<2.1.0,>=2.0.0, but you'll hav

  Found existing installation: tensorboard 2.0.0
    Uninstalling tensorboard-2.0.0:
      Successfully uninstalled tensorboard-2.0.0


In [1]:
import tensorflow as tf

In [2]:
import tensorflow.keras as keras

In [9]:
#importing the necessary libraries
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

#models that we are bringing in


In [4]:
tf.__version__

'2.1.0'

In [4]:
#test to check that tensorflow is using gpu
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [5]:
#building the architecture
#lets start with AlexNet
classifier = Sequential()


'''
    template of the networks 
    classifier.add(Convolution2D(, , strides = (), padding = "", activation = ""))
    classifier.add(MaxPooling2D(pool_size = (), strides = (), padding = ))
    classifier.add(BatchNormalization())
'''


#first convolution set
classifier.add(Convolution2D(96, 11, strides = (4,4), padding = 'valid', input_shape = (224,224,3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = "valid"))
classifier.add(BatchNormalization())

#second convolution set
classifier.add(Convolution2D(256, 11, strides = (1,1), padding = "valid", activation = "relu"))
classifier.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = "valid"))
classifier.add(BatchNormalization())

#third convolution set
classifier.add(Convolution2D(384, 3, strides = (1,1), padding = "valid", activation = "relu"))
classifier.add(BatchNormalization())

#fourth convolution set
classifier.add(Convolution2D(384, 3, strides = (1,1), padding = "valid", activation = "relu"))
classifier.add(BatchNormalization())

#fifth convolution set
classifier.add(Convolution2D(256, 3, strides = (1,1), padding = "valid", activation = "relu"))
classifier.add(MaxPooling2D(pool_size = (2,2), strides = (2,2), padding = "valid"))
classifier.add(BatchNormalization())

#flattening step
classifier.add(Flatten())

#dense network
classifier.add(Dense(units = 4096, activation = "relu"))
classifier.add(Dropout(0.4))
classifier.add(BatchNormalization())

classifier.add(Dense(units = 4096, activation = "relu"))
classifier.add(Dropout(0.4))
classifier.add(BatchNormalization())

classifier.add(Dense(units = 1000, activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(BatchNormalization())

classifier.add(Dense(units = 10, activation = "softmax"))
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 54, 54, 96)        34944     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 17, 256)       2973952   
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 256)         0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 8, 8, 256)         1024      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 6, 384)        

In [ ]:
#now to  build a custom pipeline
classifier.compile(optimizer = optimizers.SGD(lr=0.001, momentum = 0.9, decay = 0.005), loss="categorical_crossentropy", metrics=['accuracy'])
train_datagen = ImageDataGenerator(rescale= 1./255, shear_range=0.2, zoom_range=0.2, width_shift_range=0.2, height_shift_range=0.2, fill_mode='nearest')
valid_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 16
base_dir = "../dataset"

training_set = train_datagen.flow_from_directory(base_dir+'/train', target_size=(224,224),batch_size = batch_size, class_mode='categorical')
valid_set = valid_datagen.flow_from_directory(base_dir+'/train', target_size=(224,224),batch_size = batch_size, class_mode='categorical')

class_dict = training_set.class_indices
li = list(class_dict)

training_num = training_set.samples
valid_num = valid_set.samples

weight_path = '../models/best_weights.hdf5'
checkpoint = ModelCheckpoint(weight_path, monitor='val_accuracy', verbose=1, save_best_only=True,save_weights_only=True, mode='max')
callbacks_list = [checkpoint]

history = classifier.fit_generator(training_set, steps_per_epoch=training_num//batch_size, validation_data=valid_set, epochs=25, validation_steps=valid_num//batch_size, callbacks=callbacks_list)

Found 18160 images belonging to 10 classes.
Found 18160 images belonging to 10 classes.
Epoch 1/25
  56/1135 [>.............................] - ETA: 20:27 - loss: 1.7324 - accuracy: 0.4565